### Ⅰ. 라이브러리

In [61]:
import numpy as np
import pandas as pd

import os
import mido

from mido import MidiFile, MidiTrack, MetaMessage, Message




### Ⅱ. 데이터 추출
- Message
    - type
        - note_on : 음표 시작
        - control_change : 컨트롤러 변경 메시지 ex. 페달 볼륨 조절기 움직임
        - program_change : 프로그램 변경 메시지 ex. 다양한 소리 또는 패치

#### 1. pitch (음정) : [  -8192 to 8192 ]
- 음의 높낮이
- 주파수 표현

#### 2. note (음표) : [  0 to 127 ]
- msg.type = 'note_on' / 'note_off'
    - 소리의 강도 / 세기 ( velocity )
    - 지속 시간 ( time, 단위 : tick )

#### 3. control ( 컨트롤러 번호 ) [ 0 to 127 ]
- https://www.midi.org/specifications/midi-reference-tables/midi-1-0-control-change-messages-data-bytes-2
----
- control=1  ( Modulation Wheel ( 음향 변화 )) ( 0 : 왼쪽, 63 : 중앙 127 : 오른쪽 )
- control=7  ( Main Volume 조절 )
- control=10 ( Pan, 좌우 스테레오 위치 조절 )
- control=64 ( Damper Pedal on/off (Sustain) ) <= 63 Off 64<= On

In [7]:
file_name = "piano_pirate.mid"
file_path = os.path.join("midi_data", file_name)

mid = MidiFile(file_path)

In [43]:
def distribute_msg(mid_track):
    track_dict = {}
    
    track_dict['MetaMessage'] = []
    track_dict['Message'] = []
    track_dict['ETC'] = []
    
    for msg in mid_track:
        
        if isinstance(msg, MetaMessage):
            track_dict['MetaMessage'].append(msg)
            
        elif isinstance(msg, Message):
            track_dict['Message'].append(msg)
            
        else:
            track_dict['ETC'].append(msg)
            
    return track_dict

#### 4. time
- tick : mid.ticks_per_beat
    - 1박자의 tick 수를 의미
- tick_interval 을 의미

In [84]:
mid.tracks[0]

MidiTrack([
  MetaMessage('track_name', name="He's A Pirate", time=0),
  MetaMessage('text', text='By Written by Klaus Badelt', time=0),
  MetaMessage('copyright', text='Arrangement © 2004 Cerullean Pictures ', time=0),
  MetaMessage('copyright', text='Original Score © Walt Disney Pictures', time=0),
  MetaMessage('text', text='Generated by NoteWorthy Composer', time=0),
  MetaMessage('set_tempo', tempo=300000, time=0),
  MetaMessage('key_signature', key='F', time=0),
  MetaMessage('time_signature', numerator=6, denominator=8, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0),
  MetaMessage('end_of_track', time=0)])

In [83]:
print(mid.ticks_per_beat) # 1박 : print(mid.ticks_per_beat) tick

192


In [99]:
t = 192*30/60
print(t)

96.0


In [ ]:
time_in_seconds = msg.time / (ticks_per_beat * bpm / 60)

In [105]:
for idx, track in enumerate(mid.tracks):
    print(f"Track {idx + 1}: {track.name}")
    
    track_dict = distribute_msg(track)
    
    if len(track_dict['Message'])==0:
        continue;
        
    track_msg = track_dict['Message']
    
    #  msg.type == note_on 에 대해서만
    msg_type = []
    
    msg_note = []
    msg_channel = []
    msg_velocity = []
    msg_interval = []
    
    
    msg_pitch = []
    
    msg_control = []
    msg_wheel = []
    msg_pedal = []
    
    msg_time = [0] # 1분 18초 = 78초
    
    for msg in track_msg:
        
        # note off : note_on and velocity == 0
        if msg.type == 'note_on' and msg.velocity == 0:
            msg_typ = 'note_off'
        else:
            msg_typ = msg.type
            
        msg_type.append(msg_typ)
        
        time = msg.time + msg_time[-1]
        msg_time.append(time)
                
        if msg_typ == 'note_on':
            msg_note.append(msg.note)
            msg_channel.append(msg.channel)
            msg_velocity.append(msg.velocity)
            msg_interval.append(msg.time) ## 이전 이벤트와의 간격
        
        elif msg_typ == 'note_off':
            continue
            
        elif msg_typ == 'pitchwheel':
            msg_pitch.append(msg.pitch)
        
        elif msg_typ == 'control_change':
            #print(msg.control, msg.value)
            msg_control.append(msg.control)
        
            
    msg_time = msg_time[1:]
            
    
    print(np.unique(msg_type))
    print(np.unique(msg_control))
    print(msg_time[-1])

            


Track 1: He's A Pirate
Track 2: Right Hand
['control_change' 'note_off' 'note_on' 'program_change']
[ 7 10 64]
49680
Track 3: Left Hand
['control_change' 'note_off' 'note_on' 'program_change']
[ 7 10 64]
49680


In [63]:
np.unique(msg_type)

array(['control_change', 'note_on', 'program_change'], dtype='<U14')

In [32]:
track_dict = {}
track_dict['MetaMessage'] = []
track_dict['MetaMessage'].append(345)

print(track_dict)

{'MetaMessage': [345]}


In [18]:
for idx, track in enumerate(mid.tracks):
    print(f"Track {i + 1}: {track.name}")
    
    for m

Track 1: He's A Pirate
Track 2: Right Hand
Track 3: Left Hand


In [9]:
mid.tracks[0]

MidiTrack([
  MetaMessage('track_name', name="He's A Pirate", time=0),
  MetaMessage('text', text='By Written by Klaus Badelt', time=0),
  MetaMessage('copyright', text='Arrangement © 2004 Cerullean Pictures ', time=0),
  MetaMessage('copyright', text='Original Score © Walt Disney Pictures', time=0),
  MetaMessage('text', text='Generated by NoteWorthy Composer', time=0),
  MetaMessage('set_tempo', tempo=300000, time=0),
  MetaMessage('key_signature', key='F', time=0),
  MetaMessage('time_signature', numerator=6, denominator=8, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0),
  MetaMessage('end_of_track', time=0)])

In [73]:
len(mid.tracks[1])

1285

In [11]:
mid.tracks[2]

MidiTrack([
  MetaMessage('midi_port', port=0, time=0),
  MetaMessage('track_name', name='Left Hand', time=0),
  Message('program_change', channel=1, program=0, time=0),
  Message('control_change', channel=1, control=7, value=127, time=0),
  Message('control_change', channel=1, control=10, value=64, time=0),
  Message('program_change', channel=1, program=0, time=0),
  Message('control_change', channel=1, control=7, value=127, time=0),
  Message('control_change', channel=1, control=10, value=64, time=0),
  Message('control_change', channel=1, control=64, value=0, time=2304),
  Message('control_change', channel=1, control=64, value=127, time=0),
  Message('note_on', channel=1, note=26, velocity=110, time=0),
  Message('note_on', channel=1, note=38, velocity=110, time=0),
  Message('note_on', channel=1, note=26, velocity=0, time=528),
  Message('note_on', channel=1, note=38, velocity=0, time=0),
  Message('note_on', channel=1, note=26, velocity=110, time=48),
  Message('note_on', channel=

In [14]:
file_name2 = "jesu1.mid"
file_path2 = os.path.join("midi_data", file_name2)

mid2 = MidiFile(file_path2)
print(mid2)

MidiFile(type=1, ticks_per_beat=120, tracks=[
  MidiTrack([
    MetaMessage('sequencer_specific', data=(0, 0, 65), time=0),
    MetaMessage('track_name', name='JESU        ', time=0),
    MetaMessage('key_signature', key='G', time=0),
    MetaMessage('midi_port', port=0, time=0),
    MetaMessage('time_signature', numerator=9, denominator=8, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0),
    MetaMessage('set_tempo', tempo=500000, time=38340),
    MetaMessage('end_of_track', time=0)]),
  MidiTrack([
    MetaMessage('track_name', name="Jesu, Joy of Man's", time=0),
    MetaMessage('instrument_name', name='Synths Strings 1', time=0),
    MetaMessage('midi_port', port=0, time=0),
    Message('control_change', channel=0, control=0, value=80, time=0),
    Message('control_change', channel=0, control=32, value=0, time=0),
    Message('program_change', channel=0, program=50, time=0),
    Message('note_on', channel=0, note=67, velocity=104, time=60),
    Message('note_on', channel=

In [15]:
len(mid2.tracks)

8

In [16]:
mid2.tracks[0]

MidiTrack([
  MetaMessage('sequencer_specific', data=(0, 0, 65), time=0),
  MetaMessage('track_name', name='JESU        ', time=0),
  MetaMessage('key_signature', key='G', time=0),
  MetaMessage('midi_port', port=0, time=0),
  MetaMessage('time_signature', numerator=9, denominator=8, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0),
  MetaMessage('set_tempo', tempo=500000, time=38340),
  MetaMessage('end_of_track', time=0)])

In [17]:
mid2.tracks[1]

MidiTrack([
  MetaMessage('track_name', name="Jesu, Joy of Man's", time=0),
  MetaMessage('instrument_name', name='Synths Strings 1', time=0),
  MetaMessage('midi_port', port=0, time=0),
  Message('control_change', channel=0, control=0, value=80, time=0),
  Message('control_change', channel=0, control=32, value=0, time=0),
  Message('program_change', channel=0, program=50, time=0),
  Message('note_on', channel=0, note=67, velocity=104, time=60),
  Message('note_on', channel=0, note=67, velocity=0, time=48),
  Message('note_on', channel=0, note=69, velocity=104, time=12),
  Message('note_on', channel=0, note=69, velocity=0, time=48),
  Message('note_on', channel=0, note=71, velocity=104, time=12),
  Message('note_on', channel=0, note=71, velocity=0, time=48),
  Message('note_on', channel=0, note=74, velocity=104, time=12),
  Message('note_on', channel=0, note=74, velocity=0, time=48),
  Message('note_on', channel=0, note=72, velocity=104, time=12),
  Message('note_on', channel=0, note=7

In [ ]:
// pending